# Intro

In this project, we'll be working with data from the [S&P500 Index](https://en.wikipedia.org/wiki/S%26P_500_Index). 

We can use historical data on the price of the S&P500 Index to make predictions about future prices. Predicting whether an index will go up or down will help us forecast how the stock market as a whole will perform. Since stocks tend to correlate with how well the economy as a whole is performing, it can also help us make economic forecasts.

## The Data

We'll be working with the `sphist.csv` file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. 

The columns of the dataset are:

 - `Date` -- The date of the record.
 - `Open` -- The opening price of the day (when trading starts).
 - `High` -- The highest trade price during the day.
 - `Low` -- The lowest trade price during the day.
 - `Close` -- The closing price for the day (when trading is finished).
 - `Volume` -- The number of shares traded.
 - `Adj Close` -- The daily closing price, adjusted retroactively to include any corporate actions. Read more here.

We'll be using this dataset to develop a predictive model. We'll train the model with data from `1950-2012`, and try to make predictions from `2013-2015`.

Let's start by loading necessary models and the data into a pandas DataFrame.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from datetime import datetime

df = pd.read_csv('sphist.csv')
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [23]:
df.shape

(16590, 7)

Next we'll convert the `Date`column to a date:

In [24]:

df["Date"] = pd.to_datetime(df["Date"])
df.sort_values(by='Date', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

## Generating Indicators

Datasets taken from the stock market need to be handled differently than datasets from other sectors when it comes time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential, and each observation comes a day after the previous observation. Thus, the observations are not all independent, and you can't treat them as such.

Here are the indicators that we'll generate for each row:

 - The average price from the past `5` days.
 - The average price for the past `30` days.
 - The average price for the past `365` days.
 - The ratio between the average price for the past 5 days and the average price for the past 365 days.
 - The standard deviation of the price over the past 5 days.
 - The standard deviation of the price over the past 365 days.
 - The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.



In [25]:
#Add features that could be helpful for machine learning
df['5 Days Open'] = df['Open'].rolling(center=False, window=5).mean()
df['5 Days High'] = df['High'].rolling(center=False, window=5).mean()
df['5 Days Low'] = df['Low'].rolling(center=False, window=5).mean()
df['5 Days Volume'] = df['Volume'].rolling(center=False, window=5).mean()

df['Year'] = df['Date'].apply(lambda x: x.year)

#Adding Day of week column and set it to categorical
df['DOW'] = df['Date'].apply(lambda x: x.weekday())
dow_df = pd.get_dummies(df['DOW'])
df = pd.concat([df, dow_df], axis=1)
df = df.drop(['DOW'], axis=1)

In [26]:
#Shift the columns by one
df['5 Days Open'] = df['5 Days Open'].shift(1)
df['5 Days High'] = df['5 Days High'].shift(1)
df['5 Days Low'] = df['5 Days Low'].shift(1)
df['5 Days Volume'] = df['5 Days Volume'].shift(1)

In [27]:
# df.dropna(axis=0, inplace=True)

## Create Test and Train DataFrames

 - Remove any rows from the DataFrame that fall before `1951-01-03`.
 - Use the dropna method to remove any rows with `NaN` values. Pass in the `axis=0` argument to drop rows.
 - Generate two new dataframes to use in making our algorithm: 
   - `train` should contain any rows in the data with a date less than `2013-01-01`. 
   - `test` should contain any rows with a date greater than or equal to `2013-01-01`.

In [28]:
df = df[df['Date'] >= datetime(year=1951, month=1, day=3)]
df.dropna(axis=0)

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year,0,1,2,3,4
250,1951-01-03,20.690001,20.690001,20.690001,20.690001,3.370000e+06,20.690001,20.360000,20.360000,20.360000,3.126000e+06,1951,0,0,1,0,0
251,1951-01-04,20.870001,20.870001,20.870001,20.870001,3.390000e+06,20.870001,20.514000,20.514000,20.514000,3.268000e+06,1951,0,0,0,1,0
252,1951-01-05,20.870001,20.870001,20.870001,20.870001,3.390000e+06,20.870001,20.628000,20.628000,20.628000,3.358000e+06,1951,0,0,0,0,1
253,1951-01-08,21.000000,21.000000,21.000000,21.000000,2.780000e+06,21.000000,20.726001,20.726001,20.726001,3.324000e+06,1951,1,0,0,0,0
254,1951-01-09,21.120001,21.120001,21.120001,21.120001,3.800000e+06,21.120001,20.840001,20.840001,20.840001,3.192000e+06,1951,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16585,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2088.579980,2093.966064,2080.503955,3.207544e+09,2015,0,1,0,0,0
16586,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2087.283984,2095.518066,2080.811963,3.232372e+09,2015,0,0,1,0,0
16587,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2090.741992,2097.548047,2082.175977,3.245514e+09,2015,0,0,0,1,0
16588,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2089.023975,2095.948047,2073.385962,3.536224e+09,2015,0,0,0,0,1


In [29]:
# Create train and test dataframes
train = df[df["Date"] < datetime(year=2013, month=1, day=1)]
test = df[df["Date"] >= datetime(year=2013, month=1, day=1)]

## Making Predictions



In [30]:
features = ['5 Days Open', '5 Days Volume', '5 Days High', '5 Days Low', 'Year', 0, 1, 2, 3, 4]

In [31]:
lr = LinearRegression()
lr.fit(train[features], train['Close'])
predictions = lr.predict(test[features])

In [32]:
mae = mean_absolute_error(test['Close'] ,predictions)

In [33]:
print(df.tail(1))
print(mae)

            Date         Open         High          Low        Close  \
16589 2015-12-07  2090.419922  2090.419922  2066.780029  2077.070068   

             Volume    Adj Close  5 Days Open  5 Days High   5 Days Low  \
16589  4.043820e+09  2077.070068  2081.507959  2096.058057  2066.807983   

       5 Days Volume  Year  0  1  2  3  4  
16589   4.085838e+09  2015  1  0  0  0  0  
14.60031672337951


## Next Steps

We can now predict the S&P500 (with some error). We can improve the error of this model significantly, though. Think about some indicators that might be helpful to compute.

Here are some ideas that might be helpful:

 - The average volume over the past five days.
 - The average volume over the past year.
 - The ratio between the average volume for the past five days, and the average volume for the past year.
 - The standard deviation of the average volume over the past five days.
 - The standard deviation of the average volume over the past year.
 - The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.
 - The year component of the date.
 - The ratio between the lowest price in the past year and the current price.
 - The ratio between the highest price in the past year and the current price.
 - The year component of the date.
 - The day of week.
 - The day component of the date.
 - The number of holidays in the prior month.
 
 
We can also make significant structural improvements to the algorithm, and pull in data from other sources.

 - Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.

- You can also improve the algorithm used significantly. Try other techniques, like a random forest, and see if they perform better.
 - You can also incorporate outside data, such as the weather in New York City (where most trading happens) the day before, and the amount of Twitter activity around certain stocks.
 - You can also make the system real-time by writing an automated script to download the latest data when the market closes, and make predictions for the next day.
 - Finally, you can make the system "higher-resolution". You're currently making daily predictions, but you could make hourly, minute-by-minute, or second by second predictions. This will require obtaining more data, though. 
 - You could also make predictions for individual stocks instead of the S&P500.